<a href="https://colab.research.google.com/github/pushyag1/NLPClass/blob/master/genism_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Use the ‘abcnews-date-text.csv’ file.
• Load the data

• Use data_text = data[[‘headline_text’]], ‘data_text[‘index’]=data_text.index’,
‘documents=data_text’.

• Import the following libraries:
o import gensim
o from gensim.utils import simple_preprocess
o from gensim.parsing.preprocessing import STOPWORDS
o from nltk.stem import WordNetLemmatizer, SnowballStemmer
o from nltk.stem.porter import *
o import numpy as np

• Load ‘wordnet’

• Tokenize and lemmatize the text

• Create a dictionary as ‘dictionary=genism.corpora.Dictionary(processed_docs)’

• Filter extremes from dictionary

• Create a ‘bow_corpus’ using ‘doc2bow(doc)’ for doc in processed document

• From genism import corpora, models and use Tfidf for the ‘bow_corpus’

• Create a lda_model as ‘gensim.models.LdaMulticore(bow_corpus,
num_topics=10, id2word=dictionary, passes=2, workers=2)’

• Print the topics using BOW

• Run LDA using lda_model_tfidf as ‘gensim.models.LdaMulticore(corpus_tfidf,
num_topics=10, id2word=dictionary, passes=2, workers=4)’

• Print the topics using Tfidf

• Evaluate the performance by classifying sample document using LDA BOW

• Evaluate the performance by classifying sample document using LDA Tfidf

• Test the model on unseen document

In [1]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [19]:
print(len(documents))
print(documents[:5])

1186018
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [20]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [22]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [23]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [24]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [25]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [26]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3567, 1), (3568, 1)]

In [27]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 838 ("want") appears 1 time.
Word 3567 ("compulsori") appears 1 time.
Word 3568 ("ratepay") appears 1 time.


In [29]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5850076620505259),
 (1, 0.38947256567331934),
 (2, 0.4997099083387053),
 (3, 0.5063271308533074)]


In [30]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [31]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.027*"death" + 0.019*"canberra" + 0.015*"hospit" + 0.010*"water" + 0.010*"flood" + 0.010*"hobart" + 0.010*"reveal" + 0.009*"find" + 0.009*"take" + 0.008*"home"
Topic: 1 
Words: 0.023*"donald" + 0.019*"china" + 0.016*"island" + 0.016*"australian" + 0.015*"rise" + 0.013*"street" + 0.013*"fall" + 0.012*"show" + 0.010*"wall" + 0.010*"road"
Topic: 2 
Words: 0.030*"elect" + 0.021*"south" + 0.018*"live" + 0.016*"tasmania" + 0.013*"australia" + 0.012*"interview" + 0.012*"perth" + 0.011*"student" + 0.011*"school" + 0.010*"stori"
Topic: 3 
Words: 0.027*"market" + 0.018*"miss" + 0.016*"indigen" + 0.015*"victoria" + 0.013*"sydney" + 0.012*"beat" + 0.012*"citi" + 0.012*"search" + 0.011*"share" + 0.011*"price"
Topic: 4 
Words: 0.024*"charg" + 0.020*"murder" + 0.019*"australia" + 0.018*"melbourn" + 0.016*"world" + 0.015*"court" + 0.014*"face" + 0.014*"alleg" + 0.013*"test" + 0.012*"accus"
Topic: 5 
Words: 0.022*"chang" + 0.017*"jail" + 0.014*"rural" + 0.013*"busi" + 0.013*"govern" +

In [32]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.011*"govern" + 0.008*"health" + 0.006*"fund" + 0.006*"feder" + 0.005*"turnbul" + 0.005*"elect" + 0.005*"say" + 0.005*"budget" + 0.005*"care" + 0.005*"juli"
Topic: 1 Word: 0.015*"polic" + 0.015*"murder" + 0.014*"charg" + 0.011*"woman" + 0.011*"alleg" + 0.010*"court" + 0.009*"death" + 0.008*"jail" + 0.008*"sentenc" + 0.008*"shoot"
Topic: 2 Word: 0.008*"friday" + 0.006*"outback" + 0.005*"malcolm" + 0.005*"territori" + 0.005*"hobart" + 0.005*"northern" + 0.005*"shorten" + 0.004*"island" + 0.004*"remot" + 0.004*"night"
Topic: 3 Word: 0.025*"trump" + 0.016*"countri" + 0.012*"hour" + 0.008*"climat" + 0.006*"korea" + 0.006*"say" + 0.006*"financ" + 0.005*"action" + 0.005*"august" + 0.005*"russia"
Topic: 4 Word: 0.012*"stori" + 0.011*"guilti" + 0.008*"plead" + 0.006*"novemb" + 0.006*"elect" + 0.005*"john" + 0.005*"reform" + 0.004*"prime" + 0.004*"growth" + 0.004*"pass"
Topic: 5 Word: 0.015*"donald" + 0.014*"interview" + 0.008*"wednesday" + 0.007*"violenc" + 0.006*"domest" + 0.00

In [33]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [34]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5729362964630127	 
Topic: 0.022*"chang" + 0.017*"jail" + 0.014*"rural" + 0.013*"busi" + 0.013*"govern" + 0.012*"break" + 0.011*"say" + 0.011*"drum" + 0.010*"climat" + 0.010*"concern"

Score: 0.32706183195114136	 
Topic: 0.015*"feder" + 0.015*"health" + 0.014*"bushfir" + 0.014*"farmer" + 0.013*"royal" + 0.012*"plan" + 0.012*"speak" + 0.011*"guilti" + 0.010*"commiss" + 0.010*"help"

Score: 0.012501263059675694	 
Topic: 0.020*"warn" + 0.020*"nation" + 0.016*"peopl" + 0.013*"farm" + 0.011*"liber" + 0.011*"victorian" + 0.011*"leader" + 0.010*"parti" + 0.010*"state" + 0.009*"weather"

Score: 0.012500643730163574	 
Topic: 0.023*"donald" + 0.019*"china" + 0.016*"island" + 0.016*"australian" + 0.015*"rise" + 0.013*"street" + 0.013*"fall" + 0.012*"show" + 0.010*"wall" + 0.010*"road"

Score: 0.01250000111758709	 
Topic: 0.027*"death" + 0.019*"canberra" + 0.015*"hospit" + 0.010*"water" + 0.010*"flood" + 0.010*"hobart" + 0.010*"reveal" + 0.009*"find" + 0.009*"take" + 0.008*"home"

Score: 

In [35]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5014711618423462	 
Topic: 0.011*"govern" + 0.008*"health" + 0.006*"fund" + 0.006*"feder" + 0.005*"turnbul" + 0.005*"elect" + 0.005*"say" + 0.005*"budget" + 0.005*"care" + 0.005*"juli"

Score: 0.2154616415500641	 
Topic: 0.012*"stori" + 0.011*"guilti" + 0.008*"plead" + 0.006*"novemb" + 0.006*"elect" + 0.005*"john" + 0.005*"reform" + 0.004*"prime" + 0.004*"growth" + 0.004*"pass"

Score: 0.19556036591529846	 
Topic: 0.020*"news" + 0.016*"market" + 0.014*"rural" + 0.007*"australian" + 0.007*"share" + 0.007*"nation" + 0.006*"busi" + 0.006*"david" + 0.006*"rugbi" + 0.005*"live"

Score: 0.012502411380410194	 
Topic: 0.025*"trump" + 0.016*"countri" + 0.012*"hour" + 0.008*"climat" + 0.006*"korea" + 0.006*"say" + 0.006*"financ" + 0.005*"action" + 0.005*"august" + 0.005*"russia"

Score: 0.012501116842031479	 
Topic: 0.008*"friday" + 0.006*"outback" + 0.005*"malcolm" + 0.005*"territori" + 0.005*"hobart" + 0.005*"northern" + 0.005*"shorten" + 0.004*"island" + 0.004*"remot" + 0.004*"night"

In [36]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.34999993443489075	 Topic: 0.039*"trump" + 0.025*"queensland" + 0.020*"crash" + 0.019*"news" + 0.017*"die"
Score: 0.18384356796741486	 Topic: 0.027*"market" + 0.018*"miss" + 0.016*"indigen" + 0.015*"victoria" + 0.013*"sydney"
Score: 0.18332363665103912	 Topic: 0.023*"donald" + 0.019*"china" + 0.016*"island" + 0.016*"australian" + 0.015*"rise"
Score: 0.18281684815883636	 Topic: 0.027*"death" + 0.019*"canberra" + 0.015*"hospit" + 0.010*"water" + 0.010*"flood"
Score: 0.01667362079024315	 Topic: 0.015*"feder" + 0.015*"health" + 0.014*"bushfir" + 0.014*"farmer" + 0.013*"royal"
Score: 0.01666937954723835	 Topic: 0.020*"warn" + 0.020*"nation" + 0.016*"peopl" + 0.013*"farm" + 0.011*"liber"
Score: 0.016669215634465218	 Topic: 0.025*"attack" + 0.019*"polic" + 0.018*"kill" + 0.016*"govern" + 0.014*"arrest"
Score: 0.016668887808918953	 Topic: 0.022*"chang" + 0.017*"jail" + 0.014*"rural" + 0.013*"busi" + 0.013*"govern"
Score: 0.016668206080794334	 Topic: 0.030*"elect" + 0.021*"south" + 0.01